# Usage

## API key

Before you can download any images (or any metadata about them) from Google StreetView, you have to register for Google’s Static StreetView API. Go to https://developers.google.com/maps/documentation/streetview/get-api-key and follow the instructions there. Be sure to also generate an URL signing secret.

## Command line utility

The quickest way of downloading StreetView images and their metadata is to use the command line utility `streetviewdownloader`:

```bash
$ streetviewdownloader --api-key ABCDEF --url-signing-key GHIJKL --extent "POLYGON((24.40 60.12, 24.45 60.11, 24.46 60.13, 24.40 60.12))"
```

## Python

For more advanced applications, such as saving the metadata only, or for integrating the StreetView data download into a workflow pipeline, you can also use Streetviewdownloader’s [Python API](reference.html).

In [1]:
# this cell is hidden from output
# it’s used to 
# - import credentials
# - hide the warnings that are displayed because 
#   of a current mismatch in geos versions between 
#   shapely and pygeos
# - set sys.path to point to the local streetviewdownloader repo

try:
    import yaml
    import os.path
    with open(os.path.expanduser("~/.config/streetviewdownloader.yaml")) as _config_file:
        _config = yaml.safe_load(_config_file)

    API_KEY = _config["API_KEY"]
    URL_SIGNING_KEY = _config["URL_SIGNING_KEY"]
    del _config
except (ImportError, FileNotFoundError, KeyError):
    try:
        import os
        API_KEY = os.environ["sv_API_KEY"]
        URL_SIGNING_KEY = os.environ["sv_URL_SIGNING_KEY"]
    except KeyError:
        API_KEY = ""
        URL_SIGNING_KEY = ""

import warnings
warnings.simplefilter("ignore")

import sys
sys.path.append("../src/")

### Define download extent and download directory

For all examples below, we’ll download data for an area in the municipality of Kirkkonummi, about 50 km west of Helsinki, Finland. We’ll save the downloaded images and/or an associated `metadata.gpkg` into a subfolder of `~/Downloads/`:

In [2]:
import shapely.wkt
import ipyleaflet

output_directory = "/home/USER/Downloads/StreetView/"
kirkkonummi = shapely.wkt.loads("POLYGON((24.4343 60.1269, 24.4478 60.1272, 24.4384 60.1225, 24.4343 60.1269))")

map1 = ipyleaflet.Map()
map1.add_layer(ipyleaflet.WKTLayer(wkt_string=kirkkonummi.wkt))
map1.fit_bounds(((kirkkonummi.bounds[1],kirkkonummi.bounds[0]),(kirkkonummi.bounds[3],kirkkonummi.bounds[2])))

map1

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Using `StreetViewDownloader()`

The simplest way of accessing the Python API is to use the `StreetViewDownloader()` class.

In [3]:
from streetviewdownloader import StreetViewDownloader

downloader = StreetViewDownloader(API_KEY, URL_SIGNING_KEY)
downloader.download(kirkkonummi, output_directory)

This downloads the metadata to `metadata.gpkg`, and all images into the output directory. Note that, by default, data is appended to an existing metadata dataset, and images existing in the directory are not downloaded again. If you want the metadata only, call `download()` with `metadata_only=True`.

Depending on the location and size of the extent, this operation can take a while. In the background, Streetviewdownloader carries out multiple steps:

1. Decide which OSM extract from either geofabrik.de or bbbike.org fits best for the extent (the smallest extract in which the extent is fully contained)
2. Download this OSM extract, and extract and clip the street network to the extent
3. Generate a point data set on this street network with at most 25 meter distance between points
4. Query the StreetView Static API for panoramas in the immediate vicinity of each point (and remove duplicates afterwards)
5. Download six images each for every panorama.

The results of the first two steps are cached for up to seven days, as they are the most expensive ones in respect to download time and processing time. They should complete fast in subsequent runs.

### Step by step

Some of these steps can be carried out individually, in order to fine-tune the results, or to save intermediary data sets.

#### 1-3. Generate equidistant points on the street network

In [4]:
from streetviewdownloader import StreetNetworkPointGenerator
point_generator = StreetNetworkPointGenerator()
points = point_generator.points_on_street_network(kirkkonummi)
points.head()

,geometry
0,POINT (24.43637 60.12480)
1,POINT (24.43647 60.12464)
2,POINT (24.43623 60.12563)
3,POINT (24.43656 60.12565)
4,POINT (24.43689 60.12566)


In [5]:
map2 = ipyleaflet.Map()
map2.add_layer(ipyleaflet.WKTLayer(wkt_string=kirkkonummi.wkt))
map2.add_layer(ipyleaflet.GeoData(geo_dataframe=points, point_style={"radius": 1, "color": "black"}))
map2.center = map1.center
map2.zoom = map1.zoom + 2

map2

Map(center=[60.12485, 24.44105], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

#### 4. Query the StreetView API for panoramas close to each point

In [6]:
from streetviewdownloader import StreetViewMetadataDownloader
metadata_downloader = StreetViewMetadataDownloader(API_KEY, URL_SIGNING_KEY)
metadata = metadata_downloader.download(points)
metadata.head()

,date,geometry,pano_id
0,2011-09-01,POINT (24.43912 60.12362),PlXovlB4YBzyej0heen1Gg
1,2009-04-01,POINT (24.44201 60.12680),B2__1tBk90fwLCrwB-v8SA
2,2009-04-01,POINT (24.44218 60.12693),Okj0BUnoMay_siKBMEslHw
3,2011-09-01,POINT (24.43624 60.12556),fPZ0jHcEIMMfybmYV0kPLA
4,2011-09-01,POINT (24.43620 60.12580),-_XoNtKLvLBGcGL2k06-bg


In [7]:
map3 = ipyleaflet.Map()
map3.add_layer(ipyleaflet.WKTLayer(wkt_string=kirkkonummi.wkt))
map3.add_layer(ipyleaflet.GeoData(geo_dataframe=metadata[["geometry"]], point_style={"radius": 0.5, "color": "black" }))
map3.center = map1.center
map3.zoom = map1.zoom + 2
map3

Map(center=[60.12485, 24.44105], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

The metadata describes all panoramas that are available inside in the extent.\

This is a good moment to estimate the costs of downloading all images (querying metadata is free). At the time of this writing, the [price for downloading one image is 0.007 USD](https://developers.google.com/maps/documentation/streetview/usage-and-billing#static-street-view). The way Streetviewdownloader obtains the images, each panorama consists of six individual images covering a field-of-view of 60°, each.

In [8]:
estimated_costs = len(metadata) * 6 * 0.007
estimated_costs

5.7540000000000004

#### 5. Download the individual images

In [9]:
from streetviewdownloader import StreetViewImageDownloader
image_downloader = StreetViewImageDownloader(API_KEY, URL_SIGNING_KEY)
image_downloader.download(metadata, output_directory)

## Output files

The downloaded images and metadata are saved into the specified output folder. 

The metadata are saved in GeoPackage format that can be opened using any GIS software, e.g. [QGIS](https://qgis.org/) or [Geopandas](https://geopandas.org/). 

The images are saved into a directory structure derived from their IDs. Each panorama’s ID is a case-sensitive (!) alphanumeric+underscore+dash string (see above for some sample IDs in the metadata table). To optimise access times when handling thousands or millions of panoramas, Streetviewdownloader creates a three-tier subfolder structure, with the top-most directory being the first character of the ID, the second directory the first two characters, the third layer of directories the full ID. Inside each panorama’s directory, the individal images are named `{ID}_{HEADING}.jpg`, where `{ID}` is the panorama ID, and `{HEADING}` one of `000`, `060`, `120`, `180`, `240`, `300` for the direction the camera pointed to. In general, all six images should exist for each panorama.

For example, an image for heading 0 (North) for the StreetView panorama with `ID = "ABCDEF123456ABC"` would be saved to `output_directory/A/AB/ABCDEF123456ABC_000.jpg`.